# Energy Conversion Component Tests

## Imports

In [1]:
import pprint
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import copy
from energysys_components.energy_conversion import ECCParameter, ECCState,EnergyConversionComponent
from pathlib import Path


## Definition of component

In [3]:
component = ECCParameter.from_yaml(Path.cwd() / Path("components/fuel_cell_PEM.yaml"))
pprint.pprint(component)

ECCParameter(name='Example PEM Fuel Cell',
             E_in_mc_type=ECarrier(name='Hydrogen',
                                   hu_kWh_kg=33.33,
                                   density_liq_kg_m3=70.79,
                                   energy_density_liq_kWh_m3=2359.4307,
                                   color='#1982a8'),
             E_in_sd1_type=ECarrier(name='Hydrogen',
                                    hu_kWh_kg=33.33,
                                    density_liq_kg_m3=70.79,
                                    energy_density_liq_kWh_m3=2359.4307,
                                    color='#1982a8'),
             E_in_sd2_type=ECarrier(name='Electricity',
                                    hu_kWh_kg=0,
                                    density_liq_kg_m3=0,
                                    energy_density_liq_kWh_m3=0,
                                    color='#ccb80e'),
             E_out_type=ECarrier(name='Electricity',
                                 hu_kWh_

In [4]:
## Initialization of component
component_state = ECCState()
pprint.pprint(component_state)

ECCState(E_bulk=0,
         P_in=0,
         E_in=0,
         P_in_mc=0,
         E_in_mc=0,
         P_in_sd1=0,
         E_in_sd1=0,
         P_in_sd2=0,
         E_in_sd2=0,
         P_out=0,
         E_out=0,
         P_loss=0,
         E_loss=0,
         P_heat=0,
         E_heat=0,
         heatup=0,
         eta=0,
         eta_mc=0,
         opex_Eur=0,
         SoH=1,
         E_balance=0)


In [5]:
C1 = EnergyConversionComponent(par=component,
                      ts=1,
                     state=component_state)
pprint.pprint(C1)

## Run simple test cases

### Constant target output load from initial state (Stationary Step)

In [6]:
# Result DataFrame Initialization
state_parms = [a for a in dir(ECCState()) if not a.startswith('__')]
df1 = pd.DataFrame(columns=state_parms)
df1.loc[0] = vars(ECCState())

# Run different stationary cases for target output load
targets = np.linspace(0, 1., 100)

for ct, t in enumerate(targets):
    # Initialization of component
    C1_state = ECCState()
    C1 = EnergyConversionComponent(par=component,
                                   ts=1,
                                   state=copy.deepcopy(C1_state))
    C1.step_action_stationary(t)
    df1.loc[ct + 1] = vars(C1.state)
    df1.loc[ct + 1, "Target"] = t

# Create traces
fig = go.Figure()

# Create traces
for cl in df1.columns:
    fig.add_trace(go.Scatter(x=df1.Target[1:], y=df1[cl][1:],
                             mode='lines',
                             name=cl))
fig.update_layout(xaxis_title="Target rel. output load [-]",
                  # yaxis_title="Y Axis Title",
                  )
fig.update_layout(
    autosize=False,
    width=800,
    height=800)
fig.show()


## Other plot

In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# Result DataFrame Initialization
state_parms = [a for a in dir(ECCState()) if not a.startswith('__')]
df1 = pd.DataFrame(columns=state_parms)
df1.loc[0] = vars(ECCState())

# Run different stationary cases for target output load



# number of time steps
ts = 150
target = 1
target_list = []
## Initialization of component
C1_state = ECCState()
C1 = EnergyConversionComponent(par=component,
                               ts=1,
                               state = C1_state)
for t in range(ts):
    if t <= 12:
        C1.step_action(target)
        target_list.append(target*C1.par.P_out_rated)
    else:
        C1.step_action(0)
        target_list.append(0)
    df1.loc[t + 1] = vars(C1.state)


fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.05)


fig.add_trace(go.Scatter(x=df1.index[:], y=target_list,line=dict(color="black"),
                                     
                                     mode='lines',
                                     name="Target Load",
                         legendgroup = '1'),
                         row=1, col=1)
fig.add_trace(go.Scatter(x=df1.index[:], y=df1["P_out"][:],
                                     
                                     mode='lines',
                                     name="Output Load",
                         legendgroup = '1'),
                         row=1, col=1)

# Create traces
for cl in df1.columns :
    if not cl[:2] =="E_" and not cl[:3]=="err"and not cl[:4]=="opex":
        if cl[0]== "P": # and (not cl=="P_heat") :
            if cl=="P_heat" or cl=="P_in":
                pass

            else: 
                fig.add_trace(go.Scatter(x=df1.index[:], y=df1[cl][:],
                         
                         mode='lines',
                         name=cl,
                         legendgroup = '2'),
                     row=2, col=1)

        else:
            fig.add_trace(go.Scatter(x=df1.index[:], y=df1[cl][:],
                         
                         mode='lines',
                         name=cl,
                                     legendgroup = '3'),
                         row=3, col=1)



fig.update_yaxes(title_text="Power [kW]",range = [0, 2100], row=1, col=1)
fig.update_yaxes(title_text="Power [kW]", range=[0, 4400], row=2, col=1)
fig.update_yaxes(title_text="[-]", range=[0, 1.05], row=3, col=1)
fig.update_xaxes(title_text="Time [Minutes]", row=3, col=1, range=[0, 22])


for r in range(3):
    fig.add_vrect(x0=0, x1=4, row=r, col=1, fillcolor="grey", opacity=0.25, line_width=0)
    #fig.add_vrect(x0=4, x1=18, row=r, col=1, fillcolor="white", opacity=0.25, line_width=0)
    fig.add_vrect(x0=18, x1=22, row=r, col=1, fillcolor="grey", opacity=0.25, line_width=0)


fig.update_layout(height=600, width=600,
                  template="simple_white",

                  font_family="Verdana",
                  font_size=16,  # Set the font size here
                  font_color="Black",
                 legend_font_size=16,
                 legend_tracegroupgap=40)
                  #title_text="Stacked Subplots with Shared X-Axes")

fig.update_layout(
    plot_bgcolor='#F2F2F2',
    paper_bgcolor='#F2F2F2',
)
fig.show()